In [1]:
!rm -rv ./*

removed './sample_data/anscombe.json'
removed './sample_data/README.md'
removed './sample_data/california_housing_test.csv'
removed './sample_data/mnist_test.csv'
removed './sample_data/california_housing_train.csv'
removed './sample_data/mnist_train_small.csv'
removed directory './sample_data'


In [2]:
#GPU info
!nvidia-smi

Fri Aug 14 10:57:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#disable additional logs
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
%tensorflow_version 1.x
!pip install gpt-2-simple

TensorFlow 1.x selected.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=9064c0b39d04307830aacbe63234f782a0d5b4959af55d37b890b61ed2cb108a
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple


In [5]:
import gpt_2_simple as gpt2

model_name = "1558M"

#Downloads all GPT-2 models
gpt2.download_gpt2(model_name="124M")
gpt2.download_gpt2(model_name="355M")
gpt2.download_gpt2(model_name="774M")
gpt2.download_gpt2(model_name="1558M")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 241Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 79.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 387Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:03, 141Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 464Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 162Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 186Mit/s]                                                       
Fetching checkpoint: 1.05Mit [00:00, 525Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 105Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 827Mit/s]                                           

In [6]:
#use 7zip to compress model files

#!apt install p7zip
#!7z a gpt-2-models.7z models/

In [7]:
#use rclone to copy model files to google drive
#from subprocess import Popen, PIPE
#import time;
#
#!umount ./gdrive
#!apt install rclone
#!rclone config
#!rclone copy gpt-2-models.7z gdrive:/ --stats 1s --verbose

In [8]:
sess = gpt2.start_tf_sess()

In [9]:
%%file prefix.txt
Poor English: Please provide me with a short brief of the design you’re looking for and that’d be nice if you could share some examples or project you did before.
Corrected English: Please provide me with a short brief of the design you’re looking for and some examples or previous projects you’ve done would be helpful.
Poor English: If I’m stressed out about something, I tend to have problem to fall asleep.
Corrected English: If I’m stressed out about something, I tend to have a problem falling asleep.
Poor English: There is plenty of fun things to do in the summer when your able to go outside.
Corrected English: There are plenty of fun things to do in the summer when you are able to go outside.
Poor English: She no went to the market.
Corrected English: She didn’t go to the market.
Poor English: Thank you for picking me as your designer. I’d appreciate it.
Corrected English: Thank you for picking me as your designer. I appreciate it.
Poor English: I is a person.
Corrected English: I am a person.
Poor English: I used to done that.
Corrected English: I used to do that.

Writing prefix.txt


In [10]:
gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/1558M/model.ckpt
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [11]:
prefix = open('prefix.txt', 'r').read().rstrip('\n');
def grammar_correct(input):
    global prefix, model_name;
    full_prefix = prefix + "\n" + "Poor English: " + input + "\nCorrected English: ";
    return gpt2.generate(sess, model_name=model_name, prefix=full_prefix, include_prefix=False, length=20, temperature=0.9, top_p=0.9, return_as_list=True)[0].replace(full_prefix, "").split("\n")[0].split("<|endoftext|>")[0].replace("Corrected English: ", "").strip().lstrip(".").lstrip(",").lstrip("'").lstrip('"');

In [12]:
print(grammar_correct("I used to swam in the ocean."));

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
ive used to swam in the ocean.


In [13]:
print(grammar_correct("She are a person."));

She is a person.


In [14]:
print(grammar_correct("I swim wit the sharks."));

I swim with the sharks.


In [15]:
print(grammar_correct(input()));